<h1> PCA with the SVD and Eigendecomposition</h1>

This is meant as a casual way to interact with some of the linear algebra concepts introduced in the course **Linear Algebra** in the [ICME Summer Workshops in Data Science](https://icme.stanford.edu/icme-summer-workshops-2021-fundamentals-data-science#LinAlg) (2021) series, taught by Professor Margot Gerritsen and Laura Lyman. 

If you have not interacted with a Jupyter notebook before, to execute a single code block (called a *cell*), press `Shift` followed by `Enter` on your machine. Keep in mind that some cells depend on variables defined in previous cells, so the code blocks should be executed in order. 

In [1]:
from   numpy import genfromtxt
import numpy as np
import math
# Optional; suppresses scientific notation
np.set_printoptions(suppress=True)

## 1. Introduction

Let's start off the morning with showing how even an *introductory* understanding of principal component analysis (pca) can help us understand some important real world examples. Say we are researching breast cancer. We are trying to understand the key characteristics of what makes a tumor cell benign (harmless) versus malignant (potentially dangerous). 

The following data from University of Wisconsin is taken from digitized images of fine needle aspirates (FNA) of a breast mass for each patient. The features describe characteristics of the cell nuclei present in the image.

Source: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

In a study, there are many traits of a tumor that we can observe and measure. The following matrix is comprised of the data for the first 50 patients, who have a mix of benign and malignant tumors.

In [2]:
dataset = genfromtxt('WisconsinDataFirst50Cases.csv', delimiter=',')

For simplicity, suppose we only consider the 9 following properties in the dataset.

1. `radius_mean`
2. `texture_mean` (standard deviation of the grayscale values in the 2D image)
3. `perimeter_mean`
4.  `area_mean`	
5. `smoothness_mean`
6. `compactness_mean`
7. `concavity_mean`
8. `concave points_mean`
9. `symmetry_mean`

In [4]:
# Removes header row and extra columns (features)
X = dataset[1:,1:10]
[m,n]     = X.shape

Recall that the procedure for pca is as follows.

1. Populate the _m_ x _n_ data matrix matrix _X_, where _m_ is the number of measurement types and _n_ is the number of explanatory variables 
2. Subtract off the mean for each measurement type (in our case, each column of the data matrix _X_)
3. Calculate the principal components and PC scores by
 * Performing an SVD on _X_ (i.e. $X = U \Sigma V^T$), or
 * Performing an eigendecomposition on the covariance matrix $C = \frac{1}{n-1} X^T X$
 
To accomplish step (2), we can use the following straight-forward code.

In [5]:
def center_matrix(X):
    [m,n]     = X.shape
    # Subtract off the mean of each feature i.e. subtract from each column the avg. of that column 
    for feature_idx in range(n):
        X[:,feature_idx] = X[:,feature_idx] - np.mean(X[:,feature_idx])
    return(X)

# Subtract off the mean of each feature i.e. subtract from each column the avg. of that column 
for feature_idx in range(n):
    X[:,feature_idx] = X[:,feature_idx] - np.mean(X[:,feature_idx])

Now we consider the first option within step (3), which involves using the SVD to factor our data matrix $X$ directly.

## 2. Using the SVD

In [6]:
[U,Sigma,VT] = np.linalg.svd(X)

The PC scores are the nonzero entries of $\Sigma$, ranked from largest to smallest. The larger the PC score, the more its corresponding principal component (its eigenvector in $V$) characterizes the data set.

In [9]:
print(Sigma)

[2066.52718752   27.79295968   23.17301195    1.37560122    0.24598001
    0.13129855    0.10029456    0.06102043    0.03562519]


The top two PC scores ($\approx 2066.527$ and $\approx 27.793$). Their eigenvectors are the first two columns of $V$. 

In [11]:
V = np.transpose(VT)
# Show first two principal components of our data
print(V[:,:2])
print(max(abs(V[:,0])))
print(max(abs(V[:,1])))

[[-0.01012525 -0.05372279]
 [-0.00220572 -0.82526377]
 [-0.06819562 -0.5606423 ]
 [-0.99761814  0.04069534]
 [ 0.00001172 -0.00053788]
 [-0.00002646 -0.00605095]
 [-0.00009217 -0.00574415]
 [-0.0000641  -0.00182303]
 [ 0.00000156 -0.0021105 ]]
0.9976181424527256
0.8252637683444004


## 3. Using the Eigendecomposition

Suppose instead we use the eigendecomposition on the so-called *covariance matrix* $C$.  That is,

$$C := \frac{1}{n-1} X^T X = V \Lambda V^T  $$

Note that the $\frac{1}{n-1}$ factor is absorbed into $\Lambda$. The PC scores $\sigma_i$ of $X$ are related to the eigenvalues $\lambda_i$ of $C$ by the relationship 
$$\sigma_i = \sqrt{\lambda_i}$$ 

(As a technical aside, we know that the eigenvalues of $C$ are guaranteed to be non-negative, since $C$ will always be what we call a *symmetric positive semi-definite matrix*). We can compute the top two PC scores now.

In [13]:
C               = np.transpose(X).dot(X)
Lambda, V_eig   = np.linalg.eig(C)
# Be careful! While the diagonal entries of Lambda are in descending 
# order (per the convention) in this example, in general `np.linalg.eig`
# does *not* guarantee that the Lambda_ii are ordered 
Lambda

array([4270534.61676703,     772.44860773,     536.98848276,
             1.8922787 ,       0.06050616,       0.01723931,
             0.010059  ,       0.00372349,       0.00126915])

In [16]:
# From looking at Lambda, we see that the top two PC scores are its first two entries
sigma_1_eig = math.sqrt(Lambda[0])
sigma_2_eig = math.sqrt(Lambda[1])
print(sigma_1_eig)
print(sigma_2_eig)

2066.5271875218655
27.792959679276983


These are the same singular values (PC scores) $\sigma_1, \sigma_2$ found in the previous section by the SVD!

The top two principal components are then the first two vectors of `V_eig`.

In [17]:
# Show first two principal components of our data
print(V_eig[:,:2])

[[-0.01012525  0.05372279]
 [-0.00220572  0.82526377]
 [-0.06819562  0.5606423 ]
 [-0.99761814 -0.04069534]
 [ 0.00001172  0.00053788]
 [-0.00002646  0.00605095]
 [-0.00009217  0.00574415]
 [-0.0000641   0.00182303]
 [ 0.00000156  0.0021105 ]]


This is the same result as before, at least for the first two principal components!

Notice however that Python returned the second column of `V_eig` to be the 2nd column of `V_SVD` scaled by $-1$. This is okay; remember that eigenvectors are only concerned with *direction* and not scaling factors; those are handled by the PC scores $\sigma_i$. 

## 4. Interpretation 
 * For the first principal component, we can see that it is dominated by its 4th entry with magnitude `0.99761814` (where we note that the maximum magnitude of any entry in $V$ can be 1, since $V$ is an orthogonal matrix). This means that the first principal component  is best described by the 4th feature, which is `area_mean` i.e. the average 2D area of the breast mass in the digitized images.

* For the second principal component, observe that it is biased toward the second and third variables (entries `-0.82526377` and `-0.560642`3, resp.), with all remaining values at least an order of magnitude smaller. This means the second principal component  &mdash; or our new characteristic describing these masses  &mdash; is a linear combination of `texture_mean` and `perimeter_mean`. In terms of the real world, what tumor characteristic describes a combination of `texture_mean` and `perimeter_mean`?

* The variable `texture_mean` is the standard deviation of the grayscale values in the image, since topogrophy (or texture) is indicated by darker or ligher shading. Since this involves standard deviation, it means that a breast mass with a large `texture_mean` isn't necessarily large or tall; rather, its texture is non-uniform. Heuristically, we can imagine this as "spikey-ness", or an ill-defined and inconsistent border in its 3rd (vertical) dimension.

* What about the variable `perimeter_mean`? Of course, a tumor with a larger area will necessarily have a larger perimeter. However, for the characteristic the second principal component represents, we know that `perimeter_mean` indicates information that is *somewhat independent* of the tumor's area; otherwise, the fourth entry of this eigenvector (which corresponds to `area_mean`) would be larger in magnitude. So how can we vary the size of a tumor's perimeter while *minimally* affecting its area? 

* One way to increase the perimeter without increasing the area is to give the mass tall and skinny spikes protruding from its border; the perimeter is increased by traveling up and down the spikes, while minimal area is gained since the spikes are so thin. In this sense, `perimeter_mean` on its own can be a metric for how much zigzagging occurs on the breast mass's border.

* Therefore, the second principal component could be described as "overall spikey-ness or fuziness of the tumor's border."

**Principal component analysis tells us these characteristics directly**. Again, this is real data, and the interpretation here is not biased by prior medical knowledge. However, if we then Google defining characteristics for determining benign or malignant breast tumors, we can see right away that two important hallmarks are a *tumor's size* (i.e. its area) and whether it has an *irregular shape with spiky or fuzzy edges*.

Note that the top PC score ($\approx 2066.527$) is an order of magnitude larger than all the other singular values. This means that, while "overall spikey-ness or fuziness" the second-most important attribute, ultimately the first principal component (average tumor area) has the most influence over characterizing the dataset as a whole.
